In [9]:
# src/experiments/extract_my_params.py

import os
import glob
import h5py
import numpy as np
import pandas as pd


# ---------------------------------------------------------------------------
#  Parameter extraction helpers
#  (If you already implemented these in src/parameters, you can replace these
#   definitions with imports from those modules.)
# ---------------------------------------------------------------------------

def compute_tdrift99_single(wf, tp0, frac=0.999):
    """
    TDrift99 in *samples*: from tp0 to first time the waveform reaches
    `frac` of its maximum after tp0.
    """
    n = len(wf)
    start = int(tp0)
    if start >= n - 1:
        return np.nan

    segment = wf[start:]
    peak_val = segment.max()
    if peak_val <= 0:
        return np.nan

    threshold = frac * peak_val
    above = np.where(segment >= threshold)[0]
    if len(above) == 0:
        return np.nan

    return float(above[0])  # in samples


def pole_zero_correct(wf, tau_samples=500.0):
    """
    Simple pole–zero correction (high-pass IIR). This doesn't have to be
    perfect — it just needs to flatten tails reasonably for TFR.
    """
    alpha = np.exp(-1.0 / tau_samples)
    y = np.zeros_like(wf, dtype=np.float64)
    prev_x = wf[0]
    prev_y = 0.0
    for i in range(1, len(wf)):
        x = float(wf[i])
        y_i = x - prev_x + alpha * prev_y
        y[i] = y_i
        prev_x = x
        prev_y = y_i
    return y


def compute_tfr_single(wf, tp0, tail_offset=200, tail_len=600):
    """
    Tail Flattening Ratio (TFR) = std(tail_raw) / std(tail_pz)
    tail is taken starting at tp0 + tail_offset, for length tail_len (or to end).
    """
    n = len(wf)
    start = int(tp0) + tail_offset
    if start >= n - 10:  # not enough tail
        return np.nan

    end = min(n, start + tail_len)
    tail_raw = wf[start:end].astype(np.float64)

    wf_pz = pole_zero_correct(wf)
    tail_pz = wf_pz[start:end]

    std_raw = np.std(tail_raw)
    std_pz = np.std(tail_pz)

    if std_pz <= 0:
        return np.nan
    return float(std_raw / std_pz)


def smooth_gaussian(x, sigma=2.0):
    """Simple 1D Gaussian smoothing using convolution."""
    if sigma <= 0:
        return x.astype(np.float64)
    radius = int(3 * sigma)
    idx = np.arange(-radius, radius + 1, dtype=np.float64)
    kernel = np.exp(-0.5 * (idx / sigma) ** 2)
    kernel /= kernel.sum()
    padded = np.pad(x, radius, mode="edge")
    conv = np.convolve(padded, kernel, mode="same")
    return conv[radius:-radius]


def compute_peak_count_single(
    wf,
    tp0,
    window_after_tp0=400,
    grad_threshold_frac=0.05,
    min_separation=5,
):
    """
    Peak Count on the gradient:
    - baseline-subtract and normalize waveform
    - smooth
    - compute gradient
    - count local maxima above grad_threshold
    """
    n = len(wf)
    tp0 = int(tp0)

    # Baseline: first 200 samples (guard against short)
    base_end = min(200, n)
    baseline = float(np.mean(wf[:base_end]))
    wf_bs = wf - baseline

    # Normalize by global max to make threshold comparable across events
    max_val = np.max(np.abs(wf_bs))
    if max_val <= 0:
        return 0
    wf_norm = wf_bs / max_val

    # Only look near and after tp0
    start = max(tp0 - 10, 0)
    end = min(tp0 + window_after_tp0, n)
    segment = wf_norm[start:end]

    # Smooth and gradient
    seg_smooth = smooth_gaussian(segment, sigma=2.0)
    grad = np.gradient(seg_smooth)

    # Threshold relative to max gradient
    gmax = np.max(np.abs(grad))
    if gmax <= 0:
        return 0
    threshold = grad_threshold_frac * gmax

    # Count local maxima above threshold with minimum separation
    count = 0
    last_peak_idx = -min_separation - 1
    for i in range(1, len(grad) - 1):
        if grad[i] > grad[i - 1] and grad[i] > grad[i + 1] and grad[i] >= threshold:
            if i - last_peak_idx >= min_separation:
                count += 1
                last_peak_idx = i

    return int(count)


def compute_gradient_baseline_noise_single(wf, baseline_region=(0, 200)):
    """
    Gradient Baseline Noise = RMS of gradient in a pre-rise baseline window.
    """
    start, end = baseline_region
    start = max(start, 0)
    end = min(end, len(wf))
    if end - start < 5:
        return np.nan

    segment = wf[start:end].astype(np.float64)
    grad = np.gradient(segment)
    return float(np.sqrt(np.mean(grad ** 2)))


def compute_band_power_ratio_single(
    wf,
    fs=100e6,
    low_band=(0.1e6, 1e6),
    high_band=(1e6, 10e6),
):
    """
    Band Power Ratio (BPR) = power_high / power_low using FFT of the waveform.
    """
    x = wf.astype(np.float64)
    x = x - np.mean(x)

    # Real FFT
    fft_vals = np.fft.rfft(x)
    psd = np.abs(fft_vals) ** 2
    freqs = np.fft.rfftfreq(len(x), d=1.0 / fs)

    low_mask = (freqs >= low_band[0]) & (freqs < low_band[1])
    high_mask = (freqs >= high_band[0]) & (freqs < high_band[1])

    power_low = psd[low_mask].sum()
    power_high = psd[high_mask].sum()

    if power_low <= 0:
        return np.nan
    return float(power_high / power_low)


# ---------------------------------------------------------------------------
#  Per-file processing
# ---------------------------------------------------------------------------

def process_hdf5_file(h5_path, out_dir):
    """
    Read one HDF5 file, compute all 5 parameters, and write a CSV with:
      id, file, tdrift99, tfr, peak_count, gbn, bpr
    """
    print(f"Processing {h5_path}...")
    basename = os.path.basename(h5_path)

    with h5py.File(h5_path, "r") as f:
        waveforms = f["raw_waveform"][:]          # shape: (N, 3800)
        tp0 = f["tp0"][:]                         # shape: (N,)
        ids = f["id"][:]                          # shape: (N,)

    n_events = waveforms.shape[0]
    print(f"  Found {n_events} waveforms.")

    tdrift_list = []
    tfr_list = []
    peak_count_list = []
    gbn_list = []
    bpr_list = []

    for i in range(n_events):
        wf = waveforms[i]

        t0 = tp0[i]
        tdrift_list.append(compute_tdrift99_single(wf, t0))
        tfr_list.append(compute_tfr_single(wf, t0))
        peak_count_list.append(compute_peak_count_single(wf, t0))
        gbn_list.append(compute_gradient_baseline_noise_single(wf))
        bpr_list.append(compute_band_power_ratio_single(wf))

        if (i + 1) % 5000 == 0:
            print(f"    Processed {i + 1}/{n_events} events...")

    df = pd.DataFrame(
        {
            "id": ids,
            "file": basename,
            "tdrift99": tdrift_list,
            "tfr": tfr_list,
            "peak_count": peak_count_list,
            "gbn": gbn_list,
            "bpr": bpr_list,
        }
    )

    os.makedirs(out_dir, exist_ok=True)
    out_name = os.path.splitext(basename)[0] + "_myparams.csv"
    out_path = os.path.join(out_dir, out_name)
    df.to_csv(out_path, index=False)
    print(f"  Saved CSV to {out_path}\n")


# ---------------------------------------------------------------------------
#  Main: loop over all train/test files
# ---------------------------------------------------------------------------

def main():
    # Notebook-safe data directory
    DATA_DIR = os.path.abspath("data")

    TRAIN_PATTERN = os.path.join(DATA_DIR, "MJD_Train*.hdf5")
    TEST_PATTERN = os.path.join(DATA_DIR, "MJD_Test*.hdf5")

    OUT_DIR_TRAIN = os.path.join(DATA_DIR, "params_train")
    OUT_DIR_TEST = os.path.join(DATA_DIR, "params_test")

    train_files = sorted(glob.glob(TRAIN_PATTERN))
    test_files = sorted(glob.glob(TEST_PATTERN))

    print("Train files:", train_files)
    print("Test files:", test_files)

    for path in train_files:
        process_hdf5_file(path, OUT_DIR_TRAIN)

    for path in test_files:
        process_hdf5_file(path, OUT_DIR_TEST)
    process_hdf5_file("../../data/MJD_Train_15.hdf5", "../../data/params_train")



if __name__ == "__main__":
    main()


Train files: []
Test files: []
Processing ../../data/MJD_Train_15.hdf5...
  Found 65000 waveforms.
    Processed 5000/65000 events...
    Processed 10000/65000 events...
    Processed 15000/65000 events...
    Processed 20000/65000 events...
    Processed 25000/65000 events...
    Processed 30000/65000 events...
    Processed 35000/65000 events...
    Processed 40000/65000 events...
    Processed 45000/65000 events...
    Processed 50000/65000 events...
    Processed 55000/65000 events...
    Processed 60000/65000 events...
    Processed 65000/65000 events...
  Saved CSV to ../../data/params_train/MJD_Train_15_myparams.csv



In [3]:
import os
print("CWD =", os.getcwd())
print("data exists?", os.path.exists("data"))
print("files in data:", os.listdir("data"))


CWD = /Users/prithvikochhar/Documents/GitHub/Majorana-Neutrino-Hunt/src/experiments
data exists? False


FileNotFoundError: [Errno 2] No such file or directory: 'data'

In [4]:
from pathlib import Path
DATA_DIR = (Path.cwd() / "data").resolve()


In [5]:
import glob, os
print("TRAIN_PATTERN =", TRAIN_PATTERN)
print("Matches:", glob.glob(TRAIN_PATTERN))


NameError: name 'TRAIN_PATTERN' is not defined

In [7]:
from pathlib import Path
import glob

DATA_DIR = (Path.cwd() / "data").resolve()   # assumes you opened VSCode at repo root
TRAIN_PATTERN = str(DATA_DIR / "MJD_Train*.hdf5")
TEST_PATTERN  = str(DATA_DIR / "MJD_Test*.hdf5")

train_files = sorted(glob.glob(TRAIN_PATTERN))
test_files  = sorted(glob.glob(TEST_PATTERN))

print("CWD:", Path.cwd())
print("DATA_DIR:", DATA_DIR)
print("Train files found:", train_files)
print("Test files found:", test_files)


CWD: /Users/prithvikochhar/Documents/GitHub/Majorana-Neutrino-Hunt/src/experiments
DATA_DIR: /Users/prithvikochhar/Documents/GitHub/Majorana-Neutrino-Hunt/src/experiments/data
Train files found: []
Test files found: []
